In [1]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=3387bd9197b3c5af3e38773f621826058a92c5fb3d181e7c140b9dbc5a966704
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import ADASYN
from PIL import Image
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet

In [3]:
# 1. Custom Dataset
class AlzheimerDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [4]:
# 2. Loading and preprocessing the data
def load_and_preprocess_data(data_dir):
    classes = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
    X = []
    y = []
    for i, class_name in enumerate(classes):
        for split in ['train', 'test']:
            class_dir = os.path.join(data_dir, split, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                img = Image.open(img_path).convert('RGB')
                X.append(img)
                y.append(i)
    return X, np.array(y)

In [5]:
import numpy as np
from imblearn.over_sampling import ADASYN
from PIL import Image

def apply_adasyn(X, y):
    # Convert PIL Images to numpy arrays
    X_arrays = [np.array(img) for img in X]
    
    # Get the shape of the images
    img_shape = X_arrays[0].shape
    
    # Reshape the arrays to 2D for ADASYN
    X_reshaped = np.array([x.flatten() for x in X_arrays])
    
    # Apply ADASYN
    adasyn = ADASYN(random_state=42)
    X_resampled, y_resampled = adasyn.fit_resample(X_reshaped, y)
    
    # Reshape back to original image shape
    X_balanced = [x.reshape(img_shape) for x in X_resampled]
    
    # Convert back to PIL Images
    X_balanced = [Image.fromarray(x.astype('uint8')) for x in X_balanced]
    
    return X_balanced, y_resampled

# The rest of the code remains the same

In [6]:
# 4. Splitting the balanced data
def split_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, stratify=y_train, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test


In [7]:
# 5. Creating the model
def create_model(num_classes=4):
    model = EfficientNet.from_pretrained('efficientnet-b2')
    in_features = model._fc.in_features
    model._fc = nn.Linear(in_features, num_classes)
    return model

In [8]:
# 6. Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=50, device='cuda'):
    best_val_loss = float('inf')
    patience = 5
    counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
        
        train_loss = train_loss / len(train_loader.dataset)
        train_acc = train_correct / train_total
        
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()
        
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_correct / val_total
        
        scheduler.step(val_loss)
        
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f'Early stopping after {epoch+1} epochs')
                break
    
    return model

In [9]:
# 7. Testing function
def test_model(model, test_loader, criterion, device='cuda'):
    model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            test_total += labels.size(0)
            test_correct += predicted.eq(labels).sum().item()
    
    test_loss = test_loss / len(test_loader.dataset)
    test_acc = test_correct / test_total
    
    print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')

In [10]:
data_dir = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
# Load and preprocess data
X, y = load_and_preprocess_data(data_dir)

In [12]:
len(X)

6400

In [15]:
len(y)

6400

In [16]:
# Apply ADASYN
X_balanced, y_balanced = apply_adasyn(X, y)

In [17]:
len(X_balanced)

12805

In [18]:
len(y_balanced)

12805

In [19]:
# Split data
X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_balanced, y_balanced)

In [20]:
# Define transforms
transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [21]:
batch_size = 32

# Create datasets
train_dataset = AlzheimerDataset(X_train, y_train, transform=transform)
val_dataset = AlzheimerDataset(X_val, y_val, transform=transform)
test_dataset = AlzheimerDataset(X_test, y_test, transform=transform)
    
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)    

In [22]:
# Create model
model = create_model().to(device)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b2-8bb594d6.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b2-8bb594d6.pth
100%|██████████| 35.1M/35.1M [00:00<00:00, 120MB/s] 


Loaded pretrained weights for efficientnet-b2


In [23]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)    

In [24]:
# Train model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler)

Epoch 1/50: 100%|██████████| 281/281 [01:04<00:00,  4.38it/s]


Epoch 1/50
Train Loss: 0.5904, Train Acc: 0.7355
Val Loss: 0.6245, Val Acc: 0.7838


Epoch 2/50: 100%|██████████| 281/281 [01:02<00:00,  4.48it/s]


Epoch 2/50
Train Loss: 0.3010, Train Acc: 0.8803
Val Loss: 1.8973, Val Acc: 0.6948


Epoch 3/50: 100%|██████████| 281/281 [01:02<00:00,  4.48it/s]


...

Epoch 15/50
Train Loss: 0.0493, Train Acc: 0.9830
Val Loss: 0.0786, Val Acc: 0.9727


Epoch 16/50: 100%|██████████| 281/281 [01:02<00:00,  4.47it/s]


Epoch 16/50
Train Loss: 0.0151, Train Acc: 0.9952
Val Loss: 0.1132, Val Acc: 0.9703


Epoch 17/50: 100%|██████████| 281/281 [01:02<00:00,  4.47it/s]


Epoch 17/50
Train Loss: 0.0925, Train Acc: 0.9702
Val Loss: 0.1877, Val Acc: 0.9493
Early stopping after 17 epochs


In [25]:
# Load best model and test
best_model = create_model().to(device)
best_model.load_state_dict(torch.load('best_model.pth'))
test_model(best_model, test_loader, criterion)

Loaded pretrained weights for efficientnet-b2


/tmp/ipykernel_30/4122861454.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load('best_model.pth'))


Test Loss: 0.0463, Test Acc: 0.9844


In [33]:
# Update the imports
from torchvision import models

# Replace the create_model function with this:
def create_model(num_classes=4):
    model = models.vgg16(pretrained=True)
    num_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_features, num_classes)
    return model

In [34]:
# Create model
model = create_model().to(device)

In [35]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)    

In [36]:
# Train model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler)

Epoch 1/50: 100%|██████████| 281/281 [01:13<00:00,  3.84it/s]


Epoch 1/50
Train Loss: 1.1763, Train Acc: 0.4378
Val Loss: 1.0281, Val Acc: 0.4965


Epoch 2/50: 100%|██████████| 281/281 [01:13<00:00,  3.84it/s]


Epoch 2/50
Train Loss: 0.9633, Train Acc: 0.5391
Val Loss: 0.7881, Val Acc: 0.6230


Epoch 3/50: 100%|██████████| 281/281 [01:13<00:00,  3.83it/s]


...

Epoch 29/50
Train Loss: 0.1013, Train Acc: 0.9662
Val Loss: 1.3291, Val Acc: 0.6214


Epoch 30/50: 100%|██████████| 281/281 [01:13<00:00,  3.84it/s]


Epoch 30/50
Train Loss: 0.1089, Train Acc: 0.9621
Val Loss: 0.3930, Val Acc: 0.8829


Epoch 31/50: 100%|██████████| 281/281 [01:13<00:00,  3.84it/s]


Epoch 31/50
Train Loss: 0.0437, Train Acc: 0.9850
Val Loss: 0.4688, Val Acc: 0.8798
Early stopping after 31 epochs


In [37]:
# Load best model and test
best_model = create_model().to(device)
best_model.load_state_dict(torch.load('best_model.pth'))
test_model(best_model, test_loader, criterion)

/tmp/ipykernel_30/4122861454.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load('best_model.pth'))


Test Loss: 0.4044, Test Acc: 0.8778
